In [2]:
import pandas as pd
from ete3 import NCBITaxa
ncbi = NCBITaxa()

## Download genomes

Download list of completed genomes from [Prokaryotic RefSeq Genomes dataset](https://www.ncbi.nlm.nih.gov/datasets/genome/?taxon=2,2157&reference_only=true&assembly_level=3:3)

Extract column with genome accessions and feed this list of acessions to this basic script that uses [ncbi-datasets-cli software](https://www.ncbi.nlm.nih.gov/datasets/docs/v2/command-line-tools/download-and-install/) to download genomes

In [1]:
!./download_genomes.sh < test_genomes.txt 2>/dev/null

GCA_030445085.2
GCF_000005845.2
GCF_000006685.1
GCF_000006765.1
GCF_000006925.2
GCF_000006945.2
GCF_000006985.1
GCF_000007025.1
GCF_000007085.1
GCF_000007125.1


Unpack downloaded genomes

In [5]:
!wc -l test_genomes.txt

10 test_genomes.txt


In [6]:
!ls Unpacked_genomes/ | wc -l

10


## Extracting the list of EF-Tu genes

Run this simple script to extract info about tuf genes from downloaded genomes, this script will create tuf_extr.tsv table

In [7]:
!./extract_tufs.sh < test_genomes.txt 2>/dev/null

## Filter out list of genes and assign taxonomy for each assembly

In [8]:
eftu = pd.read_csv('tuf_extr.tsv', sep='\t')
eftu

,Assemly_accession,Refseq_accession,start,end,strand,gene_id,gene_name,protein_id,locus_tag,product,pseudo,partial
0,GCA_030445085.2,CP124113.1,494670,495851,+,QIA01_02370,tuf,WKD00810.1,QIA01_02370,elongation factor Tu,--,--
1,GCF_000005845.2,NC_000913.3,177665,178462,-,b0158,btuF,NP_414700.1,b0158,vitamin B12 ABC transporter periplasmic bindin...,--,--
2,GCF_000005845.2,NC_000913.3,3470148,3471329,-,b3339,tufA,NP_417798.1,b3339,translation elongation factor Tu 1,--,--
3,GCF_000005845.2,NC_000913.3,4175944,4177125,+,b3980,tufB,NP_418407.1,b3980,translation elongation factor Tu 2,--,--
4,GCF_000006685.1,NC_002620.2,712354,713535,-,TC_RS03020,tuf,WP_010904371.1,TC_RS03020,elongation factor Tu,--,--
5,GCF_000006765.1,NC_002516.2,3244345,3246327,+,PA2891,atuF,NP_251581.1,PA2891,geranyl-CoA carboxylase subunit alpha,--,--
6,GCF_000006765.1,NC_002516.2,4767814,4769004,-,PA4265,tufA,NP_252955.1,PA4265,elongation factor Tu,--,--
7,GCF_000006765.1,NC_002516.2,4784319,4785509,-,PA4277,tufB,NP_252967.1,PA4277,elongation factor Tu,--,--
8,GCF_000006925.2,NC_004337.2,169320,170117,-,SF0150,yadT,NP_706106.1,SF0150,vitamin B12-transporter protein BtuF,--,--
9,GCF_000006925.2,NC_004337.2,3439425,3440606,-,SF3357,tuf,NP_709113.2,SF3357,elongation factor Tu,--,--


Download taxonomy for each assembly accession using NCBITaxa package

In [9]:
taxons = pd.read_csv('ncbi_complete_bacteria_genomes_list.tsv', sep='\t')
taxons = taxons[['Assembly Accession', 'Organism Taxonomic ID', 'Organism Name']]

def get_phylum(taxid):
    try:
        lineage = ncbi.get_lineage(taxid)
        names = ncbi.get_taxid_translator(lineage)
        ranks = ncbi.get_rank(lineage)
        phylum = [names[t] for t in lineage if ranks[t] == 'phylum']
        return phylum[0] if phylum else None
    except:
        return None

def get_class(taxid):
    try:
        lineage = ncbi.get_lineage(taxid)
        names = ncbi.get_taxid_translator(lineage)
        ranks = ncbi.get_rank(lineage)
        classes = [names[t] for t in lineage if ranks[t] == 'class']
        return classes[0] if classes else None
    except:
        return None

taxons["Phylum"] = taxons["Organism Taxonomic ID"].apply(get_phylum)
taxons["Class"] = taxons["Organism Taxonomic ID"].apply(get_class)

taxons.to_csv("genomes_with_phylum.tsv", sep='\t', index=False)

In [10]:
taxons

,Assembly Accession,Organism Taxonomic ID,Organism Name,Phylum,Class
0,GCF_000006945.2,99287,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria
1,GCF_000195955.2,83332,Mycobacterium tuberculosis H37Rv,Actinomycetota,Actinomycetes
2,GCF_000009045.1,224308,Bacillus subtilis subsp. subtilis str. 168,Bacillota,Bacilli
3,GCF_009035845.1,470,Acinetobacter baumannii,Pseudomonadota,Gammaproteobacteria
4,GCF_000015785.2,326423,Bacillus velezensis FZB42,Bacillota,Bacilli
...,...,...,...,...,...
6354,GCF_000497445.1,1352936,Streptomyces roseochromogenus subsp. oscitans ...,Actinomycetota,Actinomycetes
6355,GCF_900258455.1,164393,Latilactobacillus fuchuensis,Bacillota,Bacilli
6356,GCF_037672085.1,650003,Vibrio paracholerae,Pseudomonadota,Gammaproteobacteria
6357,GCF_000154965.1,463191,Streptomyces sviceus ATCC 29083,Actinomycetota,Actinomycetes


Merge tables and add length of genes column

In [11]:
eftu = eftu.merge(taxons, left_on='Assemly_accession', right_on='Assembly Accession')
eftu['Length'] = abs(eftu['start'] - eftu['end'])
eftu

,Assemly_accession,Refseq_accession,start,end,strand,gene_id,gene_name,protein_id,locus_tag,product,pseudo,partial,Assembly Accession,Organism Taxonomic ID,Organism Name,Phylum,Class,Length
0,GCA_030445085.2,CP124113.1,494670,495851,+,QIA01_02370,tuf,WKD00810.1,QIA01_02370,elongation factor Tu,--,--,GCA_030445085.2,478807,Borreliella americana,Spirochaetota,Spirochaetia,1181
1,GCF_000005845.2,NC_000913.3,177665,178462,-,b0158,btuF,NP_414700.1,b0158,vitamin B12 ABC transporter periplasmic bindin...,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,797
2,GCF_000005845.2,NC_000913.3,3470148,3471329,-,b3339,tufA,NP_417798.1,b3339,translation elongation factor Tu 1,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
3,GCF_000005845.2,NC_000913.3,4175944,4177125,+,b3980,tufB,NP_418407.1,b3980,translation elongation factor Tu 2,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
4,GCF_000006685.1,NC_002620.2,712354,713535,-,TC_RS03020,tuf,WP_010904371.1,TC_RS03020,elongation factor Tu,--,--,GCF_000006685.1,243161,Chlamydia muridarum str. Nigg,Chlamydiota,Chlamydiia,1181
5,GCF_000006765.1,NC_002516.2,3244345,3246327,+,PA2891,atuF,NP_251581.1,PA2891,geranyl-CoA carboxylase subunit alpha,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1982
6,GCF_000006765.1,NC_002516.2,4767814,4769004,-,PA4265,tufA,NP_252955.1,PA4265,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
7,GCF_000006765.1,NC_002516.2,4784319,4785509,-,PA4277,tufB,NP_252967.1,PA4277,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
8,GCF_000006925.2,NC_004337.2,169320,170117,-,SF0150,yadT,NP_706106.1,SF0150,vitamin B12-transporter protein BtuF,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,797
9,GCF_000006925.2,NC_004337.2,3439425,3440606,-,SF3357,tuf,NP_709113.2,SF3357,elongation factor Tu,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,1181


Remove everything that is not tuf gene

In [12]:
eftu = eftu[~eftu['gene_name'].isin(['btuF', 'atuF', 'yadT', 'spoIIIAH', 'spoIIQ', '--'])]
eftu

,Assemly_accession,Refseq_accession,start,end,strand,gene_id,gene_name,protein_id,locus_tag,product,pseudo,partial,Assembly Accession,Organism Taxonomic ID,Organism Name,Phylum,Class,Length
0,GCA_030445085.2,CP124113.1,494670,495851,+,QIA01_02370,tuf,WKD00810.1,QIA01_02370,elongation factor Tu,--,--,GCA_030445085.2,478807,Borreliella americana,Spirochaetota,Spirochaetia,1181
2,GCF_000005845.2,NC_000913.3,3470148,3471329,-,b3339,tufA,NP_417798.1,b3339,translation elongation factor Tu 1,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
3,GCF_000005845.2,NC_000913.3,4175944,4177125,+,b3980,tufB,NP_418407.1,b3980,translation elongation factor Tu 2,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
4,GCF_000006685.1,NC_002620.2,712354,713535,-,TC_RS03020,tuf,WP_010904371.1,TC_RS03020,elongation factor Tu,--,--,GCF_000006685.1,243161,Chlamydia muridarum str. Nigg,Chlamydiota,Chlamydiia,1181
6,GCF_000006765.1,NC_002516.2,4767814,4769004,-,PA4265,tufA,NP_252955.1,PA4265,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
7,GCF_000006765.1,NC_002516.2,4784319,4785509,-,PA4277,tufB,NP_252967.1,PA4277,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
9,GCF_000006925.2,NC_004337.2,3439425,3440606,-,SF3357,tuf,NP_709113.2,SF3357,elongation factor Tu,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,1181
10,GCF_000006925.2,NC_004337.2,4195696,4196877,+,SF4053,tuf,NP_709775.1,SF4053,elongation factor Tu,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,1181
12,GCF_000006945.2,NC_003197.2,3598312,3599493,-,STM3445,tufA,NP_462349.1,STM3445,protein chain elongation factor EF-Tu,--,--,GCF_000006945.2,99287,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria,1181
13,GCF_000006945.2,NC_003197.2,4360616,4361797,+,STM4146,tufB,NP_463015.1,STM4146,protein chain elongation factor EF-Tu,--,--,GCF_000006945.2,99287,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria,1181


Remove pesudo genes, partial genes, and genes shorter than 1000bp

In [13]:
genomes_with_true_pseudo = eftu[eftu['pseudo'] == 'TRUE'].iloc[:, 0].unique()
eftu = eftu[~eftu.iloc[:, 0].isin(genomes_with_true_pseudo)]
print(len(set(eftu['Assemly_accession'])))

genomes_with_true_partial = eftu[eftu['partial'] == 'TRUE'].iloc[:, 0].unique()
eftu = eftu[~eftu.iloc[:, 0].isin(genomes_with_true_partial)]
print(len(set(eftu['Assemly_accession'])))

genomes_short_eftu = eftu[eftu['Length'] < 1000].iloc[:, 0].unique()
eftu = eftu[~eftu.iloc[:, 0].isin(genomes_short_eftu)]

print(len(set(eftu['Assemly_accession'])))
eftu

10
10
10


,Assemly_accession,Refseq_accession,start,end,strand,gene_id,gene_name,protein_id,locus_tag,product,pseudo,partial,Assembly Accession,Organism Taxonomic ID,Organism Name,Phylum,Class,Length
0,GCA_030445085.2,CP124113.1,494670,495851,+,QIA01_02370,tuf,WKD00810.1,QIA01_02370,elongation factor Tu,--,--,GCA_030445085.2,478807,Borreliella americana,Spirochaetota,Spirochaetia,1181
2,GCF_000005845.2,NC_000913.3,3470148,3471329,-,b3339,tufA,NP_417798.1,b3339,translation elongation factor Tu 1,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
3,GCF_000005845.2,NC_000913.3,4175944,4177125,+,b3980,tufB,NP_418407.1,b3980,translation elongation factor Tu 2,--,--,GCF_000005845.2,511145,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,1181
4,GCF_000006685.1,NC_002620.2,712354,713535,-,TC_RS03020,tuf,WP_010904371.1,TC_RS03020,elongation factor Tu,--,--,GCF_000006685.1,243161,Chlamydia muridarum str. Nigg,Chlamydiota,Chlamydiia,1181
6,GCF_000006765.1,NC_002516.2,4767814,4769004,-,PA4265,tufA,NP_252955.1,PA4265,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
7,GCF_000006765.1,NC_002516.2,4784319,4785509,-,PA4277,tufB,NP_252967.1,PA4277,elongation factor Tu,--,--,GCF_000006765.1,208964,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,1190
9,GCF_000006925.2,NC_004337.2,3439425,3440606,-,SF3357,tuf,NP_709113.2,SF3357,elongation factor Tu,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,1181
10,GCF_000006925.2,NC_004337.2,4195696,4196877,+,SF4053,tuf,NP_709775.1,SF4053,elongation factor Tu,--,--,GCF_000006925.2,198214,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,1181
12,GCF_000006945.2,NC_003197.2,3598312,3599493,-,STM3445,tufA,NP_462349.1,STM3445,protein chain elongation factor EF-Tu,--,--,GCF_000006945.2,99287,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria,1181
13,GCF_000006945.2,NC_003197.2,4360616,4361797,+,STM4146,tufB,NP_463015.1,STM4146,protein chain elongation factor EF-Tu,--,--,GCF_000006945.2,99287,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria,1181


In [14]:
eftu.to_csv("eftu_w_phylum.tsv", sep='\t', index=False, decimal=',')

Create summary for organisms with clear Phylum and Class assigned

In [15]:
summary = (
    eftu.groupby(["Organism Name", "Phylum", "Class", 'Refseq_accession', 'Assemly_accession'])
    .size()
    .reset_index(name="tuf_count")
    .sort_values("tuf_count", ascending=False)
)
summary

,Organism Name,Phylum,Class,Refseq_accession,Assemly_accession,tuf_count
1,Brucella melitensis bv. 1 str. 16M,Pseudomonadota,Alphaproteobacteria,NC_003317.1,GCF_000007125.1,2
2,Caldanaerobacter subterraneus subsp. tengconge...,Bacillota,Clostridia,NC_003869.1,GCF_000007085.1,2
9,Shigella flexneri 2a str. 301,Pseudomonadota,Gammaproteobacteria,NC_004337.2,GCF_000006925.2,2
5,Escherichia coli str. K-12 substr. MG1655,Pseudomonadota,Gammaproteobacteria,NC_000913.3,GCF_000005845.2,2
8,Salmonella enterica subsp. enterica serovar Ty...,Pseudomonadota,Gammaproteobacteria,NC_003197.2,GCF_000006945.2,2
6,Pseudomonas aeruginosa PAO1,Pseudomonadota,Gammaproteobacteria,NC_002516.2,GCF_000006765.1,2
0,Borreliella americana,Spirochaetota,Spirochaetia,CP124113.1,GCA_030445085.2,1
3,Chlamydia muridarum str. Nigg,Chlamydiota,Chlamydiia,NC_002620.2,GCF_000006685.1,1
4,Chlorobaculum tepidum TLS,Chlorobiota,Chlorobiia,NC_002932.3,GCF_000006985.1,1
7,Rickettsia conorii str. Malish 7,Pseudomonadota,Alphaproteobacteria,NC_003103.1,GCF_000007025.1,1


In [16]:
summary.to_csv('tuf_summary.tsv', sep='\t', index=False, decimal=',')